In [1]:
# --- Step 1: Install dependencies ---
!pip install -q faiss-cpu langchain sentence-transformers langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [3]:
# !pip install langchain_community

In [5]:
# --- Step 2: Load documents from directory ---
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [6]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [7]:
!unzip -q new_articles.zip -d new_articles

In [8]:
import os
# Set your Gemini/Google API key here (replace YOUR_KEY)
os.environ["GOOGLE_API_KEY"] = " "


In [9]:
loader = DirectoryLoader("/content/new_articles/", glob="*.txt", loader_cls=TextLoader)
documents = loader.load()

In [10]:
print(f"Loaded {len(documents)} text files")

Loaded 21 text files


In [15]:
# --- Step 3: Split documents into chunks ---
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks")

Split into 233 chunks


In [16]:
# --- Step 4: Create Hugging Face Embeddings ---
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
import numpy as np

In [17]:
class HFEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()
    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()

hf_embed = HFEmbeddings()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
# --- Step 5: Create Vector Store using FAISS ---
from langchain_community.vectorstores import FAISS
# Build FAISS index
faiss_index = FAISS.from_documents(documents=chunks, embedding=hf_embed)
print("✅ FAISS vector database created successfully!")

✅ FAISS vector database created successfully!


In [21]:
# --- Step 6: Save and Load the FAISS Index ---
save_path = "faiss_index"
faiss_index.save_local(save_path)
print("💾 FAISS index saved locally.")


💾 FAISS index saved locally.


In [22]:
# Load again (for demonstration)
faiss_index = FAISS.load_local(save_path, hf_embed, allow_dangerous_deserialization=True)
print("🔁 FAISS index reloaded successfully!")

🔁 FAISS index reloaded successfully!


In [23]:
# --- Step 7: Create Retriever ---
retriever = faiss_index.as_retriever(search_kwargs={"k": 2})


In [24]:
# --- Step 8: Setup Gemini LLM (LangChain Google GenAI Integration) ---
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    max_retries=2
)

In [25]:
# --- Step 9: Function to Ask Questions using FAISS + Gemini ---
def ask_question_with_context(question):
    # Retrieve top-matching docs
    docs = retriever.invoke(question)

    # Combine context
    context = "\n\n".join([doc.page_content for doc in docs])

    # Build prompt
    prompt = f"""
    You are a helpful AI assistant. Use the context below to answer the question concisely and accurately.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """

    # Gemini LLM response
    response = llm.invoke(prompt)

    # Display output
    print("🧠 Answer:\n", response.content)
    print("\n📄 Sources:")
    for doc in docs:
        print("-", doc.metadata.get("source", "unknown"))


In [26]:
# --- Step 10: Try it ---
ask_question_with_context("What is ai-powered supply?")

🧠 Answer:
 The provided text does not contain information about "ai-powered supply". Instead, it discusses the CMA's review of foundational AI models, Google's AI plans, and Google's AI-powered data protection project called Checks.

📄 Sources:
- /content/new_articles/05-04-cma-generative-ai-review.txt
- /content/new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt


In [27]:
# --- Step 11: Add New Data Dynamically ---
from langchain_core.documents import Document

new_text = """
Microsoft raised $5 billion to expand its AI cloud infrastructure globally.
"""
new_doc = Document(page_content=new_text, metadata={"source": "microsoft_news.txt"})

# Add to FAISS and save again
faiss_index.add_documents([new_doc])
faiss_index.save_local(save_path)
print("✅ New document added and FAISS index updated successfully!")

✅ New document added and FAISS index updated successfully!


In [ ]:
# --- Step 12: Delete FAISS Index (if needed) ---
import shutil
shutil.rmtree(save_path, ignore_errors=True)
print("🗑️ FAISS index folder removed successfully!")

🗑️ FAISS index folder removed successfully!


Agentic RAG

In [ ]:
# Agentic RAG

In [ ]:
# 1. Simple Agent

In [28]:
# --- 🧩 Agentic RAG - Simple Agent ---

def agentic_rag_simple(question: str):
    """
    A minimal 'agentic' RAG that mimics agent behavior:
    1️⃣ Receives a user question
    2️⃣ Retrieves relevant documents from FAISS
    3️⃣ Uses Gemini to generate a grounded answer
    4️⃣ Returns answer + metadata
    """

    print(f"\n🤖 Agent: Received user question → {question}\n")

    # Step 1️⃣: Retrieve top matching documents
    docs = retriever.invoke(question)
    if not docs:
        print("⚠️ No documents found for this query.")
        return

    # Step 2️⃣: Combine document contexts
    context = "\n\n".join([doc.page_content for doc in docs])

    # Step 3️⃣: Build the final LLM prompt
    prompt = f"""
    You are an AI research assistant.
    Use ONLY the context provided below to answer the user's question concisely and accurately.
    If the context is not sufficient, say "I don't have enough information."

    Context:
    {context}

    Question:
    {question}

    Answer:
    """

    # Step 4️⃣: Call Gemini model
    response = llm.invoke(prompt)
    answer = response.content.strip()

    # Step 5️⃣: Display answer + sources
    print("🧠 Agentic Answer:\n", answer)
    print("\n📄 Sources:")
    for doc in docs:
        print("-", doc.metadata.get("source", "unknown"))

    return answer


# --- ✅ Test the Agent ---
agentic_rag_simple("How much money did Microsoft raise?")



🤖 Agent: Received user question → How much money did Microsoft raise?

🧠 Agentic Answer:
 Microsoft raised $5 billion.

📄 Sources:
- microsoft_news.txt
- /content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt


'Microsoft raised $5 billion.'

In [ ]:
# 2 .“Iterative / Corrective Agentic RAG”

# The agent will:
# 1. Retrieve top documents
# 2. Generate an answer
# 3. Ask itself “Is my answer confident and well-supported?”
# 4. If not confident, retrieve again (broader search or more docs)
# 5. Regenerate the final answer.

In [29]:
# --- Step 1: (Assume all imports, FAISS setup, retriever, and llm from your previous code are ready) ---

from langchain_core.documents import Document

# --- Step 2: Define Corrective Agent Function ---
def agentic_rag_iterative(question, max_iterations=2):
    """
    Agentic RAG - Iterative / Corrective version.
    The agent evaluates its own response and retries if confidence is low.
    """
    print(f"🤖 Starting Agentic RAG for: '{question}'")

    # Initial retrieval
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    for iteration in range(max_iterations):
        print(f"\n🔄 Iteration {iteration+1}/{max_iterations}")

        # --- Step 1: Generate initial answer ---
        prompt = f"""
        You are an intelligent RAG agent.
        Use the given context to answer accurately and confidently.

        Context:
        {context}

        Question:
        {question}

        Provide a concise and factual answer.
        """
        response = llm.invoke(prompt)
        answer = response.content.strip()

        print(f"🧠 Draft Answer: {answer}\n")

        # --- Step 2: Self-check if answer is supported by context ---
        evaluation_prompt = f"""
        Evaluate the following answer based on the given context.

        Context:
        {context}

        Answer:
        {answer}

        Does this answer seem well-supported, factual, and confident?
        Reply with "YES" if it's good enough, otherwise "NO" and briefly explain why.
        """
        eval_response = llm.invoke(evaluation_prompt)
        feedback = eval_response.content.strip()

        print(f"🧐 Self-Evaluation: {feedback}\n")

        if "YES" in feedback.upper():
            print("✅ Confidence high. Returning final answer.\n")
            break  # stop if the answer is good enough
        else:
            print("⚠️ Confidence low. Re-retrieving broader context...")
            # Expand search (more documents)
            retrieved_docs = retriever.invoke(question)
            # Add new docs to context (simulate broader search)
            context += "\n\n" + "\n\n".join([doc.page_content for doc in retrieved_docs])

    # --- Step 3: Final output ---
    print("🏁 Final Answer:\n", answer)
    print("\n📚 Sources:")
    for doc in retrieved_docs:
        print("-", doc.metadata.get("source", "unknown"))


# --- Step 3: Try It ---
agentic_rag_iterative("What new product did Apple announce?")


🤖 Starting Agentic RAG for: 'What new product did Apple announce?'

🔄 Iteration 1/2
🧠 Draft Answer: The provided context does not mention any new product announcements from Apple.

🧐 Self-Evaluation: YES

✅ Confidence high. Returning final answer.

🏁 Final Answer:
 The provided context does not mention any new product announcements from Apple.

📚 Sources:
- /content/new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
- /content/new_articles/05-05-google-i-o-2023-is-next-week-heres-what-were-expecting.txt


In [ ]:
# 3. Self-Reflective Agentic RAG

# “The LLM critiques its own reasoning and refines it.”
# This RAG type uses reflection loops — the model first generates an answer,
# then analyzes its reasoning quality, identifies flaws or missing context, and improves the answer iteratively.

In [30]:
# --- Step 1: Assume FAISS, retriever, and llm are ready from previous setup ---

def agentic_rag_self_reflective(question, max_reflections=2):
    """
    Agentic RAG - Self-Reflective version.
    The LLM critiques its own reasoning and refines the final answer iteratively.
    """
    print(f"🧠 Starting Self-Reflective RAG for: '{question}'")

    # Step 1: Retrieve top docs
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Step 2: Initial answer
    base_prompt = f"""
    You are a reasoning assistant using retrieval-augmented generation.
    Use the provided context to form your best factual answer.

    Context:
    {context}

    Question:
    {question}

    Step 1: Think through the reasoning process clearly.
    Step 2: Provide a concise answer.
    """
    response = llm.invoke(base_prompt)
    answer = response.content.strip()

    # Step 3: Reflect & refine iteratively
    for i in range(max_reflections):
        print(f"\n🔁 Reflection {i+1}/{max_reflections}")
        reflection_prompt = f"""
        Review your previous answer carefully and improve it.

        Question: {question}

        Previous Answer:
        {answer}

        Context:
        {context}

        1. Identify any logical gaps or unsupported claims.
        2. Suggest corrections or improvements if necessary.
        3. Then rewrite a final improved answer.

        Respond only with the final refined answer.
        """
        reflection_response = llm.invoke(reflection_prompt)
        refined_answer = reflection_response.content.strip()
        print(f"💭 Refined Answer: {refined_answer}\n")
        answer = refined_answer  # update answer for next reflection

    print("🏁 Final Refined Answer:\n", answer)
    print("\n📚 Sources:")
    for doc in retrieved_docs:
        print("-", doc.metadata.get("source", "unknown"))


In [32]:
# result = agentic_rag_self_reflective("What is LangChain?")
result = agentic_rag_self_reflective("What new product did Apple announce?")
result


🧠 Starting Self-Reflective RAG for: 'What new product did Apple announce?'

🔁 Reflection 1/2
💭 Refined Answer: Apple did not announce any new product in the provided context. The context focuses on a collaboration between Apple and Google regarding unwanted tracking and mentions Google's announcements of the Pixel Fold, Pixel 7a, and Pixel Tablet.


🔁 Reflection 2/2
💭 Refined Answer: Apple did not announce any new product in the provided context. The context primarily discusses a collaboration between Apple and Google on unwanted tracking and mentions Google's announcements of the Pixel Fold, Pixel 7a, and Pixel Tablet. While the context mentions Apple's AirTag, it doesn't state that Apple announced a *new* product.

🏁 Final Refined Answer:
 Apple did not announce any new product in the provided context. The context primarily discusses a collaboration between Apple and Google on unwanted tracking and mentions Google's announcements of the Pixel Fold, Pixel 7a, and Pixel Tablet. While t

In [ ]:
# 4.  Planning Agentic RAG

# “The LLM decomposes the question into subtasks, retrieves per subtask, and merges answers.”

# This is multi-step RAG reasoning — the LLM becomes a “planner”:
# 1 Breaks a complex query into smaller tasks,
# 2 Performs retrieval for each task,
# 3 Generates individual answers, and
# 4 Synthesizes a unified final answer.

In [33]:
# --- Step 1: Assume FAISS, retriever, and llm already available ---

def agentic_rag_planning(question):
    """
    Agentic RAG - Planning version.
    The LLM decomposes the question into subtasks, retrieves per subtask, and merges answers.
    """
    print(f"🤖 Starting Planning Agentic RAG for: '{question}'")

    # Step 1: Ask Gemini to decompose the query
    decompose_prompt = f"""
    Decompose the following question into 2–4 smaller, sequential subtasks
    that help to answer it completely.

    Question: {question}

    Return the subtasks as a numbered list only.
    """
    subtasks_response = llm.invoke(decompose_prompt)
    subtasks = [line.strip(" -") for line in subtasks_response.content.split("\n") if line.strip()]
    print("🪜 Subtasks Identified:\n", subtasks)

    # Step 2: Solve each subtask using retrieval + context
    answers = []
    for idx, sub in enumerate(subtasks, 1):
        print(f"\n🔍 Solving Subtask {idx}: {sub}")

        docs = retriever.invoke(sub)
        context = "\n\n".join([doc.page_content for doc in docs])

        sub_prompt = f"""
        Use the following context to answer the subtask question.

        Context:
        {context}

        Subtask:
        {sub}

        Answer concisely and factually:
        """
        sub_resp = llm.invoke(sub_prompt)
        sub_answer = sub_resp.content.strip()
        answers.append(f"Subtask {idx}: {sub_answer}")

        print(f"🧩 Subtask {idx} Answer: {sub_answer}")

    # Step 3: Merge all subtask answers into a final synthesis
    merge_prompt = f"""
    You are a synthesis assistant.
    Combine the following subtask answers into one coherent final answer.

    Question: {question}

    Subtask Answers:
    {chr(10).join(answers)}

    Write a concise and well-structured final answer:
    """
    final_response = llm.invoke(merge_prompt)
    final_answer = final_response.content.strip()

    print("\n🏁 Final Synthesized Answer:\n", final_answer)


In [35]:
result = agentic_rag_planning("What new product did Apple announce?")
result


🤖 Starting Planning Agentic RAG for: 'What new product did Apple announce?'
🪜 Subtasks Identified:
 ['1. Identify recent Apple announcement events (e.g., keynotes, press releases).', '2. For each identified event, determine the products announced.', '3. From the list of announced products, identify any that are new (i.e., not updates to existing products).']

🔍 Solving Subtask 1: 1. Identify recent Apple announcement events (e.g., keynotes, press releases).
🧩 Subtask 1 Answer: The provided text focuses on Google announcements, specifically the Pixel 7a. It does not mention any recent Apple announcement events.

🔍 Solving Subtask 2: 2. For each identified event, determine the products announced.
🧩 Subtask 2 Answer: *   **Google India Announcement (May 11th):** Next Google device (likely a smartphone, design similar to the 7).
*   **Company Launch (Last Year):** BrandGPT.

🔍 Solving Subtask 3: 3. From the list of announced products, identify any that are new (i.e., not updates to existin

In [ ]:
# 5. Tool-Augmented RAG (a.k.a. Tool-Using RAG)

# In Tool-Augmented RAG, the LLM isn’t limited to static documents or FAISS retrieval.
# Instead, it can decide dynamically whether it needs to use:

# Web search

# Database queries

# Local FAISS retrieval

# Or even a custom function (API call, calculator, etc.)

# This gives the RAG system adaptive intelligence — it can expand its knowledge beyond your local vector DB
# whenever it detects missing context.

In [36]:
# --- Step 1: Install & Import Tools ---
!pip install -q ddgs
from ddgs import DDGS  # New package replacing duckduckgo-search
import textwrap

# --- Step 2: Define the Enhanced Tool-Augmented RAG Function ---
def agentic_rag_tool_augmented(question):
    """
    Tool-Augmented RAG:
    LLM decides whether to use local FAISS retrieval or external web search for better context.
    """
    print(f"⚙️ Starting Tool-Augmented RAG for: '{question}'")

    # Step 1: Retrieve from FAISS (local knowledge)
    local_docs = retriever.invoke(question)
    local_context = "\n\n".join([doc.page_content for doc in local_docs])

    # Step 2: Ask Gemini to evaluate local context quality
    judge_prompt = f"""
    You are a context quality evaluator.
    Question: {question}

    Local Context:
    {local_context[:1500]}  # limit length for safety

    Does the local context contain enough information to answer the question?
    Reply "YES" if sufficient, "NO" if not, and give a short reason.
    """
    judge_response = llm.invoke(judge_prompt)
    judge_decision = judge_response.content.strip()

    print(f"🧐 Context Evaluation: {judge_decision}\n")

    # Step 3: If context insufficient, use Web Search
    combined_context = local_context
    if "NO" in judge_decision.upper():
        print("🌐 Context insufficient — activating DuckDuckGo web search...")

        search_snippets = []
        with DDGS() as ddgs:
            for result in ddgs.text(question, max_results=3):
                snippet = result.get("body", "")
                if snippet:
                    search_snippets.append(snippet)

        if search_snippets:
            web_context = "\n\n".join(search_snippets)
            print(f"🔍 Retrieved {len(search_snippets)} web results.")
            combined_context += f"\n\n🔹 Additional Web Info:\n{web_context}"
        else:
            print("⚠️ No useful web snippets found.")
    else:
        print("✅ Local context sufficient. Using FAISS-only context.")

    # Step 4: Final synthesis (LLM generates the answer)
    final_prompt = f"""
    Use the context (from FAISS and optionally the web) to answer clearly.

    Question:
    {question}

    Context:
    {combined_context[:4000]}

    Write a factual, concise answer. If data is uncertain, indicate it.
    """
    final_response = llm.invoke(final_prompt)
    final_answer = final_response.content.strip()

    print("\n🏁 Final Answer:\n")
    print(textwrap.fill(final_answer, width=100))

    print("\n📚 Sources:")
    for doc in local_docs:
        print("-", doc.metadata.get("source", "unknown"))

    return final_answer


# --- Step 5: Test It ---
result = agentic_rag_tool_augmented("What is LangChain?")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.6 MB/s eta 0:00:00
⚙️ Starting Tool-Augmented RAG for: 'What is LangChain?'
🧐 Context Evaluation: NO. The context mentions ChatGPT, which is related to LangChain, but it doesn't define or explain what LangChain is.

🌐 Context insufficient — activating DuckDuckGo web search...
🔍 Retrieved 3 web results.

🏁 Final Answer:

LangChain is a software framework designed to simplify the creation of applications using large
language models (LLMs). It's an open-source orchestration framework that helps integrate LLMs like
GPT-4 with external data and computation, facilitating the development of applications like chatbots
and virtual agents. It provides a standard interface for integration with other tools and end-to-end
chains for common applications.

📚 Sources:
- /content/new_articl

In [ ]:
#What is Agentic AI (in simple words)?

# Agentic AI = AI that doesn’t just answer — it can act intelligently.

# Think of it like this:
# Normal AI = “A smart student that gives you answers.”
# Agentic AI = “A smart intern that thinks, plans, takes actions, and learns from results.”

# It combines LLMs (like Gemini or GPT) with tools, reasoning, and memory to work autonomously — just like a
# human assistant.

# In Technical Terms:

# Agentic AI = AI systems that have the ability to perceive, reason, decide, and act toward a goal using tools
# , APIs, and data.

# They follow a loop:
# Perceive → Think → Act → Learn (and repeat)

In [ ]:
# Types of Agentic AI

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# | Type                                                | Definition                                                                                                                            | Example                                                                                                                   |
# | --------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------- |
# | **1️⃣ Retrieval-Augmented Generation (RAG)**        | AI retrieves data from external sources (like FAISS, Chroma, or a database) before answering — ensures factual and updated responses. | You ask “Explain Data Engineering.” AI searches your company documents → finds notes → then answers using them.           |
# | **2️⃣ Tool-Augmented Agentic AI**                   | AI decides to use **tools or APIs** (like web search, SQL DB, or calculator) to find or verify data before replying.                  | You ask “Latest revenue of Tesla.” AI sees data is missing → performs web search → gives accurate result.                 |
# | **3️⃣ Memory-Augmented Agentic AI**                 | AI **remembers** previous interactions, context, and decisions over time to improve future answers.                                   | You ask “Remind me what we discussed yesterday?” AI recalls yesterday’s chat and continues seamlessly.                    |
# | **4️⃣ Planning / Multi-step Agentic AI**            | AI breaks a big goal into **smaller sub-tasks**, executes them step by step, and merges results.                                      | You say “Summarize this document and create 3 slides.” AI plans: (1) summarize → (2) design slides → (3) generate output. |
# | **5️⃣ Self-Reflective / Self-Improving Agentic AI** | AI reviews its **own reasoning**, detects mistakes, and improves its next actions automatically.                                      | It answers a math question, double-checks its steps, finds an error, fixes it, and gives the correct result.              |


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag